In [6]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)
print(torch.__version__)

1.12.1


In [18]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in /Users/sampastoriza/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-10-7 Python-3.9.13 torch-1.12.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [19]:
images=['../data/images/id_00093f2c76f6488737325859.jpg','../data/images/id_00773a42b1415d887107663a.jpg']

In [20]:
results = model(images)
results.print()

image 1/2: 3000x4000 (no detections)
image 2/2: 3120x4160 1 bird, 1 surfboard
Speed: 108.5ms pre-process, 70.4ms inference, 0.4ms NMS per image at shape (2, 3, 480, 640)
